In [1]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''

In [19]:
from os.path import join

from tqdm.auto import tqdm, trange

from rastervision.pipeline.file_system.utils import file_to_json, json_to_file
from rastervision.core.box import Box

In [4]:
aws s3 cp s3://raster-vision-ahassan/sentinel-2-embeddings/42R_2017.json s3://raster-vision-ahassan/sentinel-2-embeddings/42RUR_2017.json

2023-07-07 19:10:39          0 
2023-07-10 18:48:30  102860543 42R_2017.json
2023-07-10 22:35:06  230914167 42R_2018.json
2023-07-10 22:35:17  227634221 42R_2019.json
2023-07-10 23:04:07  239883912 42R_2020.json
2023-07-10 23:27:41  271067017 42R_2021.json
2023-07-10 23:22:16  241133686 42R_2022.json
2023-07-10 22:34:12  118011359 42R_2023.json
2023-07-07 19:55:52        516 42R_stats.json
2023-07-07 19:10:59  172173709 B13_rn18_moco_0099_ckpt.pth


In [10]:
root_dir = 's3://raster-vision-ahassan/sentinel-2-embeddings/'
embeddings_all = {}

for year in trange(2017, 2024):
    embeddings_file = join(root_dir, f'42R_{year}.json')
    embeddings = file_to_json(embeddings_file)
    embeddings_all.update(embeddings)

print(len(embeddings_all))

  0%|          | 0/7 [00:00<?, ?it/s]

Downloading:  58%|#####8    | 128M/220M [00:05<00:03, 26.8MB/s]

Downloading:  61%|######    | 132M/217M [00:05<00:03, 27.6MB/s]

Downloading:  56%|#####5    | 127M/229M [00:05<00:04, 26.5MB/s]

Downloading:  49%|####9     | 127M/259M [00:05<00:05, 26.6MB/s]

Downloading:  54%|#####3    | 124M/230M [00:05<00:04, 25.8MB/s]

Downloading:  97%|#########6| 109M/113M [00:05<00:00, 22.8MB/s]

317


In [12]:
json_to_file(embeddings_all, join(root_dir, f'42R_2017-01-01--2023-07-10.json'))

Uploading:   1%|          | 7.50M/1.32G [00:05<15:34, 1.51MB/s]

In [14]:
json_to_file(embeddings_all, f'42R_2017-01-01--2023-07-10.json')

In [34]:
box_to_embeddings = {}
windows = next(iter(embeddings_all.values()))['windows_pixel']
windows = [Box(**w) for w in windows]
len(windows)

484

In [40]:
with tqdm(windows) as bar:
    for i, w in enumerate(bar):
        box_to_embeddings[w] = []
        for t, d_t in embeddings_all.items():
            e_w = d_t['embeddings'][i]
            box_to_embeddings[w].append({t: e_w})


  0%|          | 0/484 [00:00<?, ?it/s]

In [42]:
len(box_to_embeddings[w])

317